### Import and load data entry

In [2]:
import pandas as pd
import numpy as np
from os.path import join, exists
import os
from tqdm import tqdm
import json

from PIL import Image
from torchvision import transforms

In [2]:
data_entry = pd.read_csv("Data_Entry_2017.csv", sep=";")

C:\Users\trong\AppData\Local\Temp\ipykernel_4984\1998302472.py:1: DtypeWarning: Columns (2,3,4,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_entry = pd.read_csv("Data_Entry_2017.csv", sep=";")


In [3]:
data_entry.head()

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10,Column11,Column12
0,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],NaN
1,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
2,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
3,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
4,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN


### Process folder
Count how many images are there in each folder of dataset

In [16]:
root_path = "G:\\Code\\Dataset\\archive"

img_folder = ['images_001', 'images_002', 'images_003', 'images_004', 'images_005', 'images_006', 'images_007', 'images_008', 'images_009', 'images_010', 'images_011', 'images_012']
folder_size = {}
print(exists(root_path))

for folder in img_folder:

    folder_size[folder] = (len(os.listdir((join(root_path, join(folder, "images"))))))

print(folder_size)

True
{'images_001': 4999, 'images_002': 10000, 'images_003': 10000, 'images_004': 10000, 'images_005': 10000, 'images_006': 10000, 'images_007': 10000, 'images_008': 10000, 'images_009': 10000, 'images_010': 10000, 'images_011': 10000, 'images_012': 7121}


In [17]:
with open("folder_size.json", 'w') as f:
    json.dump(folder_size, f)

### Get labels
Saved to a json file, so don't run it again

In [30]:
label_map = {}

for labels in data_entry["Column2"].to_list()[1:]:
    for label in labels.split("|"):
        label_map[label] = None    

In [31]:
c = 0
for label in label_map:
    label_map[label] = c
    c += 1

label_map

{'Cardiomegaly': 0,
 'Emphysema': 1,
 'Effusion': 2,
 'No Finding': 3,
 'Hernia': 4,
 'Infiltration': 5,
 'Mass': 6,
 'Nodule': 7,
 'Atelectasis': 8,
 'Pneumothorax': 9,
 'Pleural_Thickening': 10,
 'Pneumonia': 11,
 'Fibrosis': 12,
 'Edema': 13,
 'Consolidation': 14}

### Re-label image 
Assign each class with an index and then label each image with one-hot vector

In [34]:
img_labels_df = data_entry[["Column1","Column2"]].drop(index=0)
img_labels_df.head()

,Column1,Column2
1,00000001_000.png,Cardiomegaly
2,00000001_001.png,Cardiomegaly|Emphysema
3,00000001_002.png,Cardiomegaly|Effusion
4,00000002_000.png,No Finding
5,00000003_000.png,Hernia


In [ ]:
# clear data entry since it's quite large, don't run if you didn't load it
import gc

del data_entry
gc.collect()

In [45]:
rows , cols = img_labels_df.shape

In [51]:
# save label to train folder
train_label_path = join(root_path, "train\\class_label")
for i in tqdm(range(rows - folder_size["images_012"])):
    one_label = np.zeros(15)
    label = img_labels_df.iloc[i,1] # row , col
    for class_ in label.split('|'):
        one_label[label_map[class_]] = 1

    img_name = img_labels_df.iloc[i,0]
    np.save(join(train_label_path,img_name), one_label)

100%|██████████| 104999/104999 [00:51<00:00, 2030.56it/s]


In [52]:
# save label to test folder
test_label_path = join(root_path, "test\\class_label")
for i in tqdm(range(rows - folder_size["images_012"], rows)):
    one_label = np.zeros(15)
    label = img_labels_df.iloc[i,1] # row , col
    for class_ in label.split('|'):
        one_label[label_map[class_]] = 1

    img_name = img_labels_df.iloc[i,0]
    np.save(join(test_label_path,img_name), one_label)

100%|██████████| 7121/7121 [00:03<00:00, 2339.78it/s]
